<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import requests
import io
import sys

from bs4 import BeautifulSoup
import pandas as pd

stdout = sys.stdout
reload(sys)
sys.setdefaultencoding('utf-8')
sys.stdout = stdout

In [2]:
raw_data = pd.read_csv("../Data/rawOCRE.csv")
raw_data.tail()

,Unnamed: 0,URI,Title,RecordId,Authority,Degree,Deity,Denomination,Dynasty,Engraver,...,Obverse Type,Object Type,Portrait,Reference,Region,Reverse Legend,Reverse Type,Year,Date Record Modified,OCRE URL
42743,42743,http://numismatics.org/ocre/id/ric.5.gall(1).5,RIC V Gallienus (joint reign) 5,ric.5.gall(1).5,Valerian|Gallienus,NaN,Mars,Antoninianus,NaN,NaN,...,"Bust of Gallienus, radiate, cuirassed, right",Coin,Gallienus,NaN,Gallia,P M TR P V COS III,"Mars, walking right, holding spear in right ha...",257,2018-08-08T16:20:41Z,http://numismatics.org/ocre/id/ric.5.gall(1).5
42744,42744,http://numismatics.org/ocre/id/ric.5.gall(1).4,RIC V Gallienus (joint reign) 4,ric.5.gall(1).4,Valerian|Gallienus,NaN,Victory,Aureus,NaN,NaN,...,"Head of Gallienus, laureate, right",Coin,Gallienus,NaN,Gallia,VICTORIAE AVGG,"Victory, winged, draped, standing in biga",258|259,2018-08-08T16:20:41Z,http://numismatics.org/ocre/id/ric.5.gall(1).4
42745,42745,http://numismatics.org/ocre/id/ric.5.gall(1).163,RIC V Gallienus (joint reign) 163,ric.5.gall(1).163,Valerian|Gallienus,NaN,Providentia,Antoninianus,NaN,NaN,...,"Head of Gallienus, radiate, right",Coin,Gallienus,NaN,Italy,PROVIDENTIA AVGG,"Providentia, draped, standing left, leaning on...",257|258,2018-08-08T16:21:11Z,http://numismatics.org/ocre/id/ric.5.gall(1).163
42746,42746,http://numismatics.org/ocre/id/ric.5.gall(1).160,RIC V Gallienus (joint reign) 160,ric.5.gall(1).160,Valerian|Gallienus,NaN,Providentia,Antoninianus,NaN,NaN,...,"Head of Gallienus, radiate, right",Coin,Gallienus,NaN,Italy,PROVIDENTIA AVGG,"Providentia, draped, standing left, holding ba...",257|258,2018-08-08T16:21:11Z,http://numismatics.org/ocre/id/ric.5.gall(1).160
42747,42747,http://numismatics.org/ocre/id/ric.5.gall(1).147,RIC V Gallienus (joint reign) 147,ric.5.gall(1).147,Valerian|Gallienus,NaN,Liberalitas,Antoninianus,NaN,NaN,...,"Bust of Gallienus, radiate, cuirassed, right",Coin,Gallienus,NaN,Italy,LIBERALITAS AVGG,"Liberalitas, draped, standing left, holding te...",254|255,2018-08-08T16:21:08Z,http://numismatics.org/ocre/id/ric.5.gall(1).147


In [3]:
#links = ['http://numismatics.org/ocre/id/ric.2.tr.797', 'http://numismatics.org/ocre/id/ric.5.gall(1).147']
data = None
for url in raw_data['OCRE URL'].values:
    # Get csv of coins from OCRE
    html_data = requests.get(url).content
    soup = BeautifulSoup(html_data, 'html.parser')
    csv_url_container = soup.find('a', {'title': 'Download CSV'})
    if csv_url_container is None:
        continue
    csv_url = csv_url_container['href']
    items = pd.read_csv(csv_url)
    
    # Create dataframe of same coins
    coins_data = None
    for index, row in items.iterrows():
        item_data = raw_data[raw_data['OCRE URL'] == url]
        item_data['URL'] = row['object']
        item_data['collection'] = row['collection']
        item_data['collectionID'] = row['identifier']
        item_data['findspot'] = row['findspot']
        item_data['hoard'] = row['hoard']
        if coins_data is None:
            coins_data = item_data
        else:
            coins_data = coins_data.append(item_data, ignore_index=True)
    
    # Add same coins to all coins dataframe
    if data is None:
        data = coins_data.copy()
    else:
        data = data.append(coins_data, ignore_index=True)

In [5]:
data.to_csv("../Data/OCRE.csv")

In [7]:
data = pd.read_csv("../Data/OCRE.csv")

In [9]:
data['hoard'].unique()

array([nan, 'http://nomisma.org/id/lliria_hoard',
       'http://nomisma.org/id/olivers_orchard_hoards',
       'http://nomisma.org/id/normanby_hoard'], dtype=object)